In [1]:
import os
import openai
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
# Fine tuning increases the accuracy from 84% to 84.20%; minor change

In [3]:
df = pd.read_csv('../data/csv/processed_toxic_classification_dataset_short.csv')
df['Toxic'] = df['Toxic'].apply(lambda x: str(int(x)))
df

,Text,Toxic
0,Geez forgetful Weve already discussed Marx ana...,0
1,Carioca RFA Thanks support request adminship f...,0
2,Birthday worries Enjoy ur daytalke,0
3,Pseudoscience category Im assuming article pse...,0
4,phrase exists would provided search engine eve...,0
...,...,...
995,2006 UTC rest us ever gone past 3RR good one V...,0
996,Yay lets Pedantic Semantics dance rolls eyes t...,0
997,supposed Know,0
998,guys really discuss napoleon need get fuckin l...,1


In [4]:
load_dotenv()
FINE_TUNED_TOXIC_DETECTION_API_KEY = os.getenv("FINE_TUNED_TOXIC_DETECTION_API_KEY")

In [5]:
fine_tuned = OpenAI(
    api_key=FINE_TUNED_TOXIC_DETECTION_API_KEY
)

In [6]:
tuned = pd.DataFrame(columns=['Text', 'Toxic'])

In [7]:
evaluation_prompt = ''
# Read the evaluation prompt from the text file with utf-8 encoding
with open("../data/text/detailed_instructions.txt", "r", encoding="utf-8") as file:
    evaluation_prompt += file.read()

In [8]:
evaluation_prompt

'Task: You are given a set of comments from social media. Your goal is to evaluate each comment and classify it as either toxic or non-toxic. A comment is considered toxic if it contains harmful, abusive, offensive, or harassing language. If a comment is toxic, label it with a 1. Otherwise, if it is neutral or polite and does not contain harmful language, label it with a 0 for non-toxic.\n\nDefinition of Toxicity:\nToxic comments (1): These are comments that include language or content that is rude, disrespectful, hateful, harmful, threatening, or harassing. This includes but is not limited to:\n\nProfanity or slurs (racial, gender-based, homophobic, etc.)\nDirect or implied threats of violence or harm\nPersonal attacks, name-calling, or derogatory statements\nEncouragement of harm or violence toward individuals or groups\nLanguage that promotes discrimination, hatred, or exclusion\nInsults or offensive jokes directed at individuals or groups based on their identity, beliefs, or charac

In [10]:
for index, row in df.iterrows():
    completion = fine_tuned.chat.completions.create(
        model="ft:gpt-4o-2024-08-06:personal::AENT6Q3Y",
        messages=[
            {
                "role": "system", 
                "content": evaluation_prompt
            },
            {
                "role": "user",
                "content": row['Text']
            }
        ]
    )
    tuned.loc[index] = [row['Text'], completion.choices[0].message.content]


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
tuned

,Text,Toxic
0,Geez forgetful Weve already discussed Marx ana...,0
1,Carioca RFA Thanks support request adminship f...,0
2,Birthday worries Enjoy ur daytalke,0
3,Pseudoscience category Im assuming article pse...,0
4,phrase exists would provided search engine eve...,0
...,...,...
995,2006 UTC rest us ever gone past 3RR good one V...,0
996,Yay lets Pedantic Semantics dance rolls eyes t...,0
997,supposed Know,0
998,guys really discuss napoleon need get fuckin l...,1


In [ ]:
compare = tuned['Toxic'] == df['Toxic']
gptfour_original_accuracy = compare.values.sum() / compare.size

In [ ]:
compare

0      True
1      True
2      True
3      True
4      True
       ... 
995    True
996    True
997    True
998    True
999    True
Name: Toxic, Length: 1000, dtype: bool

In [ ]:
# 97.30% with too mid-detailed evaluation prompt
print(f"Accuracy: {gptfour_original_accuracy * 100:.2f}%")

Accuracy: 97.00%


In [ ]:

completion = fine_tuned.chat.completions.create(
    model="ft:gpt-4o-2024-08-06:personal::AENT6Q3Y",
    messages=[
        {
            "role": "system", 
            "content": "The user has a toxicity threshold of 1, meaning that they find 90 to 100 percent of comments non-toxic and have extremely high tolerance. Label this text as either 0 for non-toxic or 1 for toxic."
        },
        {
            "role": "user",
            "content": df['Text'][998]
        }
    ]
)

In [ ]:
completion.choices[0].message.content

'1'